In [3]:
!pip install -qU langchain_core langchain_openai langchain_community langchain-qdrant qdrant-client langsmith openai tiktoken cohere lxml protobuf===3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 5.7 MB/s eta 0:00:00


In [4]:
!pip install --quiet langchain_experimental langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 1.6 MB/s eta 0:00:00


In [17]:
!pip install -qU pymupdf ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 655.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 970.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.7/185.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 952.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 677.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibi

In [5]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

Enter your OpenAI API Key:··········


In [6]:
import nest_asyncio
nest_asyncio.apply()

In [35]:
from langchain_community.document_loaders import PyMuPDFLoader

ai_framework_document = PyMuPDFLoader(file_path="https://nvlpubs.nist.gov/nistpubs/ai/NIST.AI.600-1.pdf").load() #22,204 words, 64 pages
ai_blueprint_document = PyMuPDFLoader(file_path="https://www.whitehouse.gov/wp-content/uploads/2022/10/Blueprint-for-an-AI-Bill-of-Rights.pdf").load() #31,393 words, 73 pages

#### 🟢 **Default Chunking Strategy**

`semantic chunking`

#### 🟢 **Experimental Chunking Strategy**

`semantic chunking` (initial) then `fixed chunk size` + `overlap`

#### 🟢 **Rationale**

I chose semantic chunking because meaning is important. By combining semantic chunking with fixed chunk size and overlaps helps ensure balanced and retain/retrieve contextual information better than using the default chunking strategy alone.

#### 🟢 **Additional notes**
Using my Experimental Chunking Strategy, plus TE3-large for chunking and embedding generation, have produced the best quality of response so far. (See comparison with base RAG below)

In [89]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings

# Selecting TE3-large because the combined documents is 53,597 words long and is 137 pages
text_splitter = SemanticChunker(OpenAIEmbeddings(model="text-embedding-3-large"))
base_text_splitter = SemanticChunker(OpenAIEmbeddings())

In [79]:
def metadata_generator(document, name):
  """
  Adds metadata name to the semantically chunked document
  """
  collection = text_splitter.split_documents(document)
  for doc in collection:
    doc.metadata["source"] = name
  return collection

In [80]:
# The next chunking step after semantic chunking
fixed_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

recursive_framework_document = fixed_text_splitter.split_documents(metadata_generator(ai_framework_document, "AI Framework"))
recursive_blueprint_document = fixed_text_splitter.split_documents(metadata_generator(ai_blueprint_document, "AI Blueprint"))

In [90]:
# Since both documents have metadata, we can now combine them before vectorization
combined_documents = recursive_framework_document + recursive_blueprint_document
base_combined_documents = metadata_generator(ai_framework_document, "AI Framework") + metadata_generator(ai_blueprint_document, "AI Blueprint")

In [91]:
from langchain_openai import OpenAIEmbeddings

# Selecting TE3-large because the combined documents is 53,597 words long and is 137 pages
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [95]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=combined_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Policy"
)

base_vectorstore = Qdrant.from_documents(
    documents=base_combined_documents,
    embedding=OpenAIEmbeddings(),
    location=":memory:",
    collection_name="Base AI Policy"
)


In [97]:
retriever = vectorstore.as_retriever()
base_retriever = base_vectorstore.as_retriever()

In [107]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [109]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [110]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [113]:
from IPython.display import HTML, display


output1 = rag_chain.invoke({"question" : "What is the AI framework all about?"})
output2 = base_rag_chain.invoke({"question" : "What is the AI framework all about?"})

html = f"""
<table>
    <tr>
        <th style="text-align: left;">Response from my strategy</th>
        <th style="text-align: left;">Response from Base RAG</th>
    </tr>
    <tr>
        <td style="text-align: left;">{output1}</td>
        <td style="text-align: left;">{output2}</td>
    </tr>
</table>
"""

display(HTML(html))

Response from my strategy,Response from Base RAG
"The AI framework is centered on the evaluation and development of AI products, services, and systems. It is being developed by the National Institute of Standards and Technology (NIST) through a consensus-driven, open, transparent, and collaborative process. The framework aims to foster innovative approaches to address various characteristics of trustworthiness, including accuracy, explainability, interpretability, reliability, privacy, robustness, safety, security (resilience), and the mitigation of unintended and/or harmful biases and uses. It also considers principles such as transparency, accountability, and fairness throughout the lifecycle of AI technologies and systems, from pre-design to deployment, use, and evaluation. The framework is expected to be released in the winter of 2022-23.","The AI framework mentioned in the provided context is primarily concerned with managing the risks posed by artificial intelligence (AI) to individuals, organizations, and society. Specifically: 1. **NIST AI Risk Management Framework**: Developed by the National Institute of Standards and Technology (NIST), this framework aims to incorporate trustworthiness considerations into the design, development, use, and evaluation of AI products, services, and systems. It focuses on characteristics such as accuracy, explainability, interpretability, reliability, privacy, robustness, safety, security, and mitigation of unintended and/or harmful bias and uses. The framework is being developed through a consensus-driven, open, transparent, and collaborative process. 2. **Blueprint for an AI Bill of Rights**: This is a set of five principles and associated practices intended to guide the design, use, and deployment of automated systems to protect the rights of the American public. It was developed through extensive public consultation and aims to align AI systems with democratic values and protect civil rights, civil liberties, and privacy. 3. **Executive Order 13960**: This order mandates that certain federal agencies adhere to nine principles when designing, developing, acquiring, or using AI, ensuring that AI is lawful, purposeful, accurate, safe, understandable, responsible, regularly monitored, transparent, and accountable. Overall, the AI framework seeks to promote the safe and effective use of AI technologies while ensuring they are trustworthy and aligned with ethical and democratic values."
